<a href="https://colab.research.google.com/github/Marilyn-S/Machine-Learning-Final/blob/main/MLProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
# Get data stored in GitHub Repo
url = 'https://raw.githubusercontent.com/Marilyn-S/Machine-Learning-Final/main/data/datasets_tech_mental_health.csv'
df = pd.read_csv(url)

In [ ]:
df.head()

In [ ]:
df.tail()